In [ ]:
import zipfile 
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
data_set = "dogs-vs-cats"
with zipfile.ZipFile("/kaggle/input/"+ data_set +"/train.zip","r") as z:
    z.extractall(".")
    destination = '/kaggle/files/images/train'
    z.extractall(destination)
    
with zipfile.ZipFile("/kaggle/input/"+ data_set +"/test1.zip","r") as z:
    z.extractall(".")
    destination = '/kaggle/files/images/test'
    z.extractall(destination)

In [ ]:
def list_full_paths(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory)]

In [ ]:
train = pd.DataFrame({'filepath': list_full_paths('/kaggle/files/images/train/train')})
train['truth_label'] = np.where(train['filepath'].str.contains('dog'), 'dog', 'cat')

test = pd.DataFrame({'filepath': list_full_paths('/kaggle/files/images/test/test1')})

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train['truth_label'].value_counts()

In [ ]:
# Splitting train data into train and test
X_train, X_test = train_test_split(train, test_size=0.2)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_dataframe(dataframe=X_train, 
                                                 x_col='filepath', 
                                                 y_col='truth_label', 
                                                 class_mode='categorical', 
                                                 target_size = (224, 224), 
                                                 batch_size = 128)

test_set = test_datagen.flow_from_dataframe(dataframe=X_test, 
                                            x_col='filepath', 
                                            y_col='truth_label', 
                                            class_mode='categorical', 
                                            target_size = (224, 224), 
                                            batch_size = 128)

In [ ]:
train[train['truth_label'] == "cat"].head(1)

In [ ]:
train[train['truth_label'] == "dog"].head(1)

In [ ]:
# Displaying the Image
plt.subplot(1, 2, 1)
# Dog Image
image_dog=cv2.imread("/kaggle/files/images/train/train/dog.890.jpg")
plt.imshow(image_dog)
plt.subplot(1, 2, 2)
# Cat Image
image_cat=cv2.imread("/kaggle/files/images/train/train/cat.3660.jpg")
plt.imshow(image_cat)
plt.show()

### Creating VGG16 Architecture From Scratch

In [ ]:
model = Sequential()
# Convolution Layer 1
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
# Max Pooling Layer 1
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Convolution Layer 2
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# Max Pooling Layer 2
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Convolution Layer 3
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# Max Pooling Layer 3
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Convolution Layer 4
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# Max Pooling Layer 4
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# Convolution Layer 5
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
# Max Pooling Layer 5
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
# Flatten Layer
model.add(Flatten(name='flatten'))
# 3 Dense Layers
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dense(2, activation='sigmoid', name='output'))

In [ ]:
model.summary()

In [ ]:
Vgg16 = Model(inputs=model.input, outputs=model.get_layer('vgg16').output)

In [ ]:
Vgg16.load_weights("/kaggle/input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [ ]:
for layer in Vgg16.layers:
    layer.trainable = False

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
model.compile(loss="binary_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
history = model.fit(training_set, validation_data=test_set, epochs=10)

In [ ]:
# Training loss and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, 11)

plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'b',label='training_loss')
plt.plot(epochs, val_loss, 'r', label='valid_loss')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
# Training accuracy and validation accuracy
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.figure(figsize=(10, 6))
plt.plot(epochs, accuracy, 'b',label='train_accuracy')
plt.plot(epochs, val_accuracy, 'r', label='val_accuracy')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
y_true = []
y_pred = []

In [ ]:
len(test_set)

In [ ]:
for i in range(40):
    x, y = test_set[i]
    y_true.extend([np.argmax(i) for i in y])
    y_pred.extend([np.argmax(i) for i in model.predict(x)])

In [ ]:
len(y_true), len(y_pred)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
confusion_matrix(y_true, y_pred)

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
from tensorflow.keras.preprocessing import image
from keras.applications.resnet import preprocess_input

In [ ]:
def predict_class(img_path):
    img=image.load_img(img_path,target_size=(224,224))
    x=image.img_to_array(img)
    x=np.expand_dims(x, axis=0)
    img_data=preprocess_input(x)
    classes=model.predict(img_data)
    return classes

In [ ]:
predict_class("/kaggle/files/images/train/train/cat.3660.jpg")

### Pre-Trained Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(2, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
history = model.fit(training_set, validation_data=test_set, epochs=10)

In [ ]:
# Training loss and validation loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, 11)

plt.figure(figsize=(10, 5))
plt.plot(epochs, loss, 'b',label='training_loss')
plt.plot(epochs, val_loss, 'r', label='valid_loss')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
# Training accuracy and validation accuracy
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.figure(figsize=(10, 6))
plt.plot(epochs, accuracy, 'b',label='train_accuracy')
plt.plot(epochs, val_accuracy, 'r', label='val_accuracy')
plt.xlabel('epochs')
plt.legend()
plt.show()

In [ ]:
y_true = []
y_pred = []

In [ ]:
for i in range(40):
    x, y = test_set[i]
    y_true.extend([np.argmax(i) for i in y])
    y_pred.extend([np.argmax(i) for i in model.predict(x)])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(y_true, y_pred)

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
predict_class("/kaggle/files/images/train/train/cat.3660.jpg")